# 5. Formális nyelvek és automaták

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## 1. Automata kézzel

In [ ]:
# állapotátmenetek
d = {
    0:{'b':1},
    1:{'e':2},
    2:{'e':3},
    3:{'e':3, '!':4},
    4:{}
}


# automata
def fsa(transitions, start, accepts, text):
    state = start
    for char in text:
        state = transitions[state].get(char)
        if state == None:
            return False
    return state in accepts


# teszt
for tok in ['beee!', 'be!', 'beee!!', 'xxx']:
    res = fsa(d, 0, {4}, tok)
    print(tok, res, sep='\t')

## 2. NLTK

*NLTK*: Natural Language Toolkit, NLP eszközöket és erőforrások pythonos gyűjteménye. Elsősorban tanítási és prototípuskészítési célra.

Erőforrások kezelése

In [ ]:
import nltk

# nltk.download();

### 2.1. Elemzés RegexpTagger-rel

Az első illeszkedő szabály lesz a nyertes $\to$ a sorrend számít!

In [ ]:
from nltk import RegexpTagger


patterns = [
    (r'.*k$', 'Pl'),
    (r'.*t$', 'Acc'),
    (r'.*', 'Nom')
]


re_tagger = RegexpTagger(patterns)
s = ['kutya', 'kutyát', 'kutyák']


# tsv
for i in re_tagger.tag(s):
    print(i[0], i[1], sep='\t')

### 2.2. Elemzés CFG nyelvtannal

#### Nyelvtan megadása

* Terminális: python-stringek
* Nemterminális: idézőjel nélkül
* Szabály: egy sor, `->` választja el a szabály két oldalát

In [ ]:
%%bash
cat np.cfg

#### Nyelvtan tesztelése

In [ ]:
from nltk.parse import TestGrammar

test1 = {}
test1['doc'] = 'határozatlan névelő'
test1['accept'] = ['egy macska', 'egy kutya']
test1['reject'] = ['macska egy']

test2 = {}
test2['doc'] = 'határozott névelő'
test2['accept'] = ['a macska', 'a kutya']
test2['reject'] = ['macska a']

testsuite = [test1, test2]
tester = TestGrammar('np.cfg', testsuite)
tester.run()

#### Környezetfüggetlen nyelvtan (CFG)

In [ ]:
from nltk import CFG

npgram = CFG.fromstring(open('np.cfg').read())

#### Elemző létrehozása nyelvtanhoz

Egy szöveg elemzésére sok több módszer van. Az NLTK is tartalmaz jópárat. Az egyik legegyszerűbb, *top-down* elemző a *Recursive Descent Parser*.

In [ ]:
from nltk.parse import RecursiveDescentParser

rdp = RecursiveDescentParser(npgram)

#### Elemzés

In [ ]:
t1 = list(rdp.parse('a macska'.split()))
t2 = list(rdp.parse('a egy'.split()))

print(t1)
print(t2)

#### Fák

In [ ]:
t = t1[0]

In [ ]:
# típus
print(type(t))

In [ ]:
# ágrajz
t.pretty_print()

In [ ]:
# alfák
for tree in list(t.subtrees()):
    print(tree)

In [ ]:
# levelek címkéi
print(t.pos())

In [ ]:
# tsv
for i in t.pos():
    print(i[0], i[1], sep='\t')

### Házi feladat (2b)

Az NP-felismerőt kéne bővíteni az alábbiak szerint:

* Kezelje a mellékneves szerkezeteket is (*a barna kutya*, *egy szép barna macska*).
* Fogadja el a névelő nélküli szerkezeteket is (*kutya*, *szép macska*, *nagy barna kutya*).
* Fogadja el a tulajdonneveket (*Péter*, *Mari*).


A feladat megoldásához

1. Írjunk elfogadási és elutasítási teszteket, amik lefedik a fenti jelenségeket!
2. Módosítsuk az `np.cfg` fájlt (írjuk meg a megfelelő szabályokat, vegyünk fel pár mellék- és tulajdonnevet) úgy, hogy átmenjen a tesztjeinken.
3. Írjunk egy elemző-függvényt (`np_parser()`), ami egy sztringet vár bemenetként, és egy listát ad vissza, ami az egyes szavak és elemzéseik tuple-jeit tartalmazza (pl. `[('a', 'Det'), ('kutya', 'N')]`). A melléknevek `A`, a tulajdonnevek `Prop` címkét kapjanak. A megoldásnál feltehetjük, hogy a bemeneteknek legfeljebb egy elemzése lehetséges, vagyis a többértelműséget nem kell kezelnünk, de ha nincs elemzés, azt igen!

## 3. Irodalom

* [NLTK book](http://www.nltk.org/book/)
    * CFG: 8.4
    * Tagger-ek: 5.4
* [NLTK HOWTOs](http://www.nltk.org/howto/)
    * [Parsing](http://www.nltk.org/howto/parse.html)